In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))


TensorFlow version: 2.20.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# List GPUs
gpus = tf.config.list_physical_devices('GPU')
print("GPUs:", gpus)

# Test if TF can place operations on GPU
with tf.device('/GPU:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)

print("Matrix multiplication successful on GPU")


TensorFlow version: 2.20.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Matrix multiplication successful on GPU


I0000 00:00:1757387504.301869    6777 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21755 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1757387504.303116    6777 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22148 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:05:00.0, compute capability: 8.9


In [1]:
import os
import numpy as np

# Update with your dataset paths
DRONE_ROOT   = r"/home/destrox-907/Husnian's FYP/Dataset/MFCC Drone Dataset/MFCC Drone Dataset"
NODRONE_ROOT = r"/home/destrox-907/Husnian's FYP/Dataset/MFCC Noise Dataset/MFCC Noise Dataset"

def check_npy_files(root_dir, log_file):
    bad_files = []
    total, ok, fail = 0, 0, 0

    for r, _, files in os.walk(root_dir):
        for f in files:
            if f.lower().endswith(".npy"):
                total += 1
                path = os.path.join(r, f)
                try:
                    arr = np.load(path, allow_pickle=False)
                    ok += 1
                except Exception as e:
                    print(f"[CORRUPTED] {path} -> {e}")
                    bad_files.append(path)
                    fail += 1
    
    print(f"\nSummary for {root_dir}:")
    print(f" Total: {total} | OK: {ok} | Corrupted: {fail}")

    with open(log_file, "a") as f:
        for bf in bad_files:
            f.write(bf + "\n")

# Run checks
if __name__ == "__main__":
    log_file = "bad_files.txt"
    if os.path.exists(log_file):
        os.remove(log_file)

    check_npy_files(DRONE_ROOT, log_file)
    check_npy_files(NODRONE_ROOT, log_file)

    print("\nDiagnostics complete ✅")
    print("If there are corrupted files, they are listed in bad_files.txt")



Summary for /home/destrox-907/Husnian's FYP/Dataset/MFCC Drone Dataset/MFCC Drone Dataset:
 Total: 190462 | OK: 190462 | Corrupted: 0

Summary for /home/destrox-907/Husnian's FYP/Dataset/MFCC Noise Dataset/MFCC Noise Dataset:
 Total: 123286 | OK: 123286 | Corrupted: 0

Diagnostics complete ✅
If there are corrupted files, they are listed in bad_files.txt


In [2]:
from pathlib import Path
import xml.etree.ElementTree as ET

# Input / output
xml_file = "/home/destrox-907/Downloads/Offer_description.xml"   # your input file
txt_file = "/home/destrox-907/Downloads/description.txt"

# Namespaces we need
NS = {"pkg": "http://schemas.microsoft.com/office/2006/xmlPackage",
      "w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}

# Parse the XML
tree = ET.parse(xml_file)
root = tree.getroot()

# Find the /word/document.xml part
doc_text = []
for part in root.findall("pkg:part", NS):
    if part.get("{%s}name" % NS["pkg"]) == "/word/document.xml":
        xmlData = part.find("pkg:xmlData", NS)
        if xmlData is not None:
            document = xmlData.find("w:document", NS)
            if document is not None:
                for para in document.findall(".//w:p", NS):
                    texts = [t.text for t in para.findall(".//w:t", NS) if t.text]
                    if texts:
                        doc_text.append("".join(texts))

# Save to .txt
Path(txt_file).write_text("\n".join(doc_text), encoding="utf-8")
print(f"✅ Saved extracted text to {txt_file}")


✅ Saved extracted text to /home/destrox-907/Downloads/description.txt
